In [1]:
# %matplotlib widget

from __future__ import annotations

import re
from collections import defaultdict
from dataclasses import dataclass, field
from itertools import permutations, product
from math import inf
from random import choice

import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import numpy.typing as npt
from mpl_toolkits.mplot3d import axes3d
from numpy import int_, object_
from numpy.typing import NDArray
from test_utilities import test
from util import *

COLORS = list(mcolors.CSS4_COLORS.keys())

<link href="style.css" rel="stylesheet"></link>
<article class="day-desc"><h2>--- Day 12: Garden Groups ---</h2><p>Why not search for the Chief Historian near the <a href="/2023/day/5">gardener</a> and his <a href="/2023/day/21">massive farm</a>? There's plenty of food, so The Historians grab something to eat while they search.</p>
<p>You're about to settle near a complex arrangement of garden plots when some Elves ask if you can lend a hand. They'd like to set up <span title="I originally wanted to title this puzzle &quot;Fencepost Problem&quot;, but I was afraid someone would then try to count fenceposts by mistake and experience a fencepost problem.">fences</span> around each region of garden plots, but they can't figure out how much fence they need to order or how much it will cost. They hand you a map (your puzzle input) of the garden plots.</p>
<p>Each garden plot grows only a single type of plant and is indicated by a single letter on your map. When multiple garden plots are growing the same type of plant and are touching (horizontally or vertically), they form a <em>region</em>. For example:</p>
<pre><code>AAAA
BBCD
BBCC
EEEC
</code></pre>
<p>This 4x4 arrangement includes garden plots growing five different types of plants (labeled <code>A</code>, <code>B</code>, <code>C</code>, <code>D</code>, and <code>E</code>), each grouped into their own region.</p>
<p>In order to accurately calculate the cost of the fence around a single region, you need to know that region's <em>area</em> and <em>perimeter</em>.</p>
<p>The <em>area</em> of a region is simply the number of garden plots the region contains. The above map's type <code>A</code>, <code>B</code>, and <code>C</code> plants are each in a region of area <code>4</code>. The type <code>E</code> plants are in a region of area <code>3</code>; the type <code>D</code> plants are in a region of area <code>1</code>.</p>
<p>Each garden plot is a square and so has <em>four sides</em>. The <em>perimeter</em> of a region is the number of sides of garden plots in the region that do not touch another garden plot in the same region. The type <code>A</code> and <code>C</code> plants are each in a region with perimeter <code>10</code>. The type <code>B</code> and <code>E</code> plants are each in a region with perimeter <code>8</code>. The lone <code>D</code> plot forms its own region with perimeter <code>4</code>.</p>
<p>Visually indicating the sides of plots in each region that contribute to the perimeter using <code>-</code> and <code>|</code>, the above map's regions' perimeters are measured as follows:</p>
<pre><code>+-+-+-+-+
|A A A A|
+-+-+-+-+     +-+
              |D|
+-+-+   +-+   +-+
|B B|   |C|
+   +   + +-+
|B B|   |C C|
+-+-+   +-+ +
          |C|
+-+-+-+   +-+
|E E E|
+-+-+-+
</code></pre>
<p>Plants of the same type can appear in multiple separate regions, and regions can even appear within other regions. For example:</p>
<pre><code>OOOOO
OXOXO
OOOOO
OXOXO
OOOOO
</code></pre>
<p>The above map contains <em>five</em> regions, one containing all of the <code>O</code> garden plots, and the other four each containing a single <code>X</code> plot.</p>
<p>The four <code>X</code> regions each have area <code>1</code> and perimeter <code>4</code>. The region containing <code>21</code> type <code>O</code> plants is more complicated; in addition to its outer edge contributing a perimeter of <code>20</code>, its boundary with each <code>X</code> region contributes an additional <code>4</code> to its perimeter, for a total perimeter of <code>36</code>.</p>
<p>Due to "modern" business practices, the <em>price</em> of fence required for a region is found by <em>multiplying</em> that region's area by its perimeter. The <em>total price</em> of fencing all regions on a map is found by adding together the price of fence for every region on the map.</p>
<p>In the first example, region <code>A</code> has price <code>4 * 10 = 40</code>, region <code>B</code> has price <code>4 * 8 = 32</code>, region <code>C</code> has price <code>4 * 10 = 40</code>, region <code>D</code> has price <code>1 * 4 = 4</code>, and region <code>E</code> has price <code>3 * 8 = 24</code>. So, the total price for the first example is <code><em>140</em></code>.</p>
<p>In the second example, the region with all of the <code>O</code> plants has price <code>21 * 36 = 756</code>, and each of the four smaller <code>X</code> regions has price <code>1 * 4 = 4</code>, for a total price of <code><em>772</em></code> (<code>756 + 4 + 4 + 4 + 4</code>).</p>
<p>Here's a larger example:</p>
<pre><code>RRRRIICCFF
RRRRIICCCF
VVRRRCCFFF
VVRCCCJFFF
VVVVCJJCFE
VVIVCCJJEE
VVIIICJJEE
MIIIIIJJEE
MIIISIJEEE
MMMISSJEEE
</code></pre>
<p>It contains:</p>
<ul>
<li>A region of <code>R</code> plants with price <code>12 * 18 = 216</code>.</li>
<li>A region of <code>I</code> plants with price <code>4 * 8 = 32</code>.</li>
<li>A region of <code>C</code> plants with price <code>14 * 28 = 392</code>.</li>
<li>A region of <code>F</code> plants with price <code>10 * 18 = 180</code>.</li>
<li>A region of <code>V</code> plants with price <code>13 * 20 = 260</code>.</li>
<li>A region of <code>J</code> plants with price <code>11 * 20 = 220</code>.</li>
<li>A region of <code>C</code> plants with price <code>1 * 4 = 4</code>.</li>
<li>A region of <code>E</code> plants with price <code>13 * 18 = 234</code>.</li>
<li>A region of <code>I</code> plants with price <code>14 * 22 = 308</code>.</li>
<li>A region of <code>M</code> plants with price <code>5 * 12 = 60</code>.</li>
<li>A region of <code>S</code> plants with price <code>3 * 8 = 24</code>.</li>
</ul>
<p>So, it has a total price of <code><em>1930</em></code>.</p>
<p><em>What is the total price of fencing all regions on your map?</em></p>
</article>


In [2]:
from collections import deque
from collections.abc import Iterator


tests = [
    {
        "name": "Small Example",
        "s": """
            AAAA
            BBCD
            BBCC
            EEEC
        """,
        "expected": 140,
    },
    {
        "name": "Example inside regions",
        "s": """
            OOOOO
            OXOXO
            OOOOO
            OXOXO
            OOOOO
        """,
        "expected": 772,
    },
    {
        "name": "Larger Example",
        "s": """
            RRRRIICCFF
            RRRRIICCCF
            VVRRRCCFFF
            VVRCCCJFFF
            VVVVCJJCFE
            VVIVCCJJEE
            VVIIICJJEE
            MIIIIIJJEE
            MIIISIJEEE
            MMMISSJEEE
        """,
        "expected": 1930,
    },
]


class Garden(str):
    def __init__(self, s: str) -> None:
        self.map = [l.strip() for l in s.strip().splitlines()]

    def price(self) -> int:
        som = 0
        for group in self.groups().values():
            area = len(group)
            perimeter = 4 * area
            inner_borders = 0
            for r, c in group:
                for rn, cn in self.neigbors(r, c):
                    if (rn, cn) in group:
                        inner_borders += 1

            perimeter -= inner_borders
            som += area * perimeter

        return som

    def groups(self) -> dict[int, list[set[int, int]]]:
        rows, cols = len(self.map), len(self.map[0])
        groups = {
            i: {(r, c)} for i, (r, c) in enumerate(product(range(rows), range(cols)))
        }
        belogs_to = {
            (r, c): i for i, (r, c) in enumerate(product(range(rows), range(cols)))
        }

        queue = deque([(0, 0)])
        seen = set()

        while queue:
            r, c = queue.popleft()

            if (r, c) in seen:
                continue

            seen.add((r, c))

            plant = self.map[r][c]
            group = belogs_to[(r, c)]

            for rn, cn in self.neigbors(r, c):
                gn = belogs_to[(rn, cn)]
                pn = self.map[rn][cn]

                if plant == pn and group != gn:
                    belogs_to[(rn, cn)] = group

                    for m in groups[gn]:
                        belogs_to[m] = group

                    groups[group] |= groups[gn]
                    del groups[gn]

                queue.append((rn, cn))

        return groups

    def neigbors(self, row: int, col: int) -> Iterator[tuple[int, int]]:
        if row - 1 >= 0:
            yield row - 1, col

        if col + 1 < len(self.map[0]):
            yield row, col + 1

        if row + 1 < len(self.map):
            yield row + 1, col

        if col - 1 >= 0:
            yield row, col - 1

    def __str__(self) -> str:
        return "\n".join(self.map)


@test(tests=tests)
def partI_test(s: str) -> int:
    g = Garden(s)
    return g.price()


Test Small Example passed, for partI_test.
Test Example inside regions passed, for partI_test.
Test Larger Example passed, for partI_test.
Success


In [3]:
with open("../input/day12.txt") as f:
    puzzle = f.read()

print(f"Part I: {Garden(puzzle).price()}")

Part I: 1446042


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>1446042</code>.</p><p class="day-success">The first half of this puzzle is complete! It provides one gold star: *</p>


<link href="style.css" rel="stylesheet"></link>
<article class="day-desc"><h2 id="part2">--- Part Two ---</h2><p>Fortunately, the Elves are trying to order so much fence that they qualify for a <em>bulk discount</em>!</p>
<p>Under the bulk discount, instead of using the perimeter to calculate the price, you need to use the <em>number of sides</em> each region has. Each straight section of fence counts as a side, regardless of how long it is.</p>
<p>Consider this example again:</p>
<pre><code>AAAA
BBCD
BBCC
EEEC
</code></pre>
<p>The region containing type <code>A</code> plants has <code>4</code> sides, as does each of the regions containing plants of type <code>B</code>, <code>D</code>, and <code>E</code>. However, the more complex region containing the plants of type <code>C</code> has <code>8</code> sides!</p>
<p>Using the new method of calculating the per-region price by multiplying the region's area by its number of sides, regions <code>A</code> through <code>E</code> have prices <code>16</code>, <code>16</code>, <code>32</code>, <code>4</code>, and <code>12</code>, respectively, for a total price of <code><em>80</em></code>.</p>
<p>The second example above (full of type <code>X</code> and <code>O</code> plants) would have a total price of <code><em>436</em></code>.</p>
<p>Here's a map that includes an E-shaped region full of type <code>E</code> plants:</p>
<pre><code>EEEEE
EXXXX
EEEEE
EXXXX
EEEEE
</code></pre>
<p>The E-shaped region has an area of <code>17</code> and <code>12</code> sides for a price of <code>204</code>. Including the two regions full of type <code>X</code> plants, this map has a total price of <code><em>236</em></code>.</p>
<p>This map has a total price of <code><em>368</em></code>:</p>
<pre><code>AAAAAA
AAABBA
AAABBA
ABBAAA
ABBAAA
AAAAAA
</code></pre>
<p>It includes two regions full of type <code>B</code> plants (each with <code>4</code> sides) and a single region full of type <code>A</code> plants (with <code>4</code> sides on the outside and <code>8</code> more sides on the inside, a total of <code>12</code> sides). Be especially careful when counting the fence around regions like the one full of type <code>A</code> plants; in particular, each section of fence has an in-side and an out-side, so the fence does not connect across the middle of the region (where the two <code>B</code> regions touch diagonally). (The Elves would have used the Möbius Fencing Company instead, but their contract terms were too one-sided.)</p>
<p>The larger example from before now has the following updated prices:</p>
<ul>
<li>A region of <code>R</code> plants with price <code>12 * 10 = 120</code>.</li>
<li>A region of <code>I</code> plants with price <code>4 * 4 = 16</code>.</li>
<li>A region of <code>C</code> plants with price <code>14 * 22 = 308</code>.</li>
<li>A region of <code>F</code> plants with price <code>10 * 12 = 120</code>.</li>
<li>A region of <code>V</code> plants with price <code>13 * 10 = 130</code>.</li>
<li>A region of <code>J</code> plants with price <code>11 * 12 = 132</code>.</li>
<li>A region of <code>C</code> plants with price <code>1 * 4 = 4</code>.</li>
<li>A region of <code>E</code> plants with price <code>13 * 8 = 104</code>.</li>
<li>A region of <code>I</code> plants with price <code>14 * 16 = 224</code>.</li>
<li>A region of <code>M</code> plants with price <code>5 * 6 = 30</code>.</li>
<li>A region of <code>S</code> plants with price <code>3 * 6 = 18</code>.</li>
</ul>
<p>Adding these together produces its new total price of <code><em>1206</em></code>.</p>
<p><em>What is the new total price of fencing all regions on your map?</em></p>
</article>


In [11]:
from pprint import pprint

from more_itertools import minmax


tests = [
    {
        "name": "Small Example",
        "s": """
            AAAA
            BBCD
            BBCC
            EEEC
        """,
        "expected": 80,
    },
    {
        "name": "Example inside regions",
        "s": """
            OOOOO
            OXOXO
            OOOOO
            OXOXO
            OOOOO
        """,
        "expected": 436,
    },
    {
        "name": "Example E-shaped regions",
        "s": """
            EEEEE
            EXXXX
            EEEEE
            EXXXX
            EEEEE
        """,
        "expected": 236,
    },
    {
        "name": "Another Example",
        "s": """
            AAAAAA
            AAABBA
            AAABBA
            ABBAAA
            ABBAAA
            AAAAAA
        """,
        "expected": 368,
    },
    {
        "name": "Larger Example",
        "s": """
            RRRRIICCFF
            RRRRIICCCF
            VVRRRCCFFF
            VVRCCCJFFF
            VVVVCJJCFE
            VVIVCCJJEE
            VVIIICJJEE
            MIIIIIJJEE
            MIIISIJEEE
            MMMISSJEEE
        """,
        "expected": 1206,
    },
]


class GardenII(Garden):
    def price(self, debug: bool = False) -> int:
        if debug:
            print(self)

        rows, cols = len(self.map), len(self.map[0])
        som = 0

        for group in self.groups().values():
            area = len(group)
            sides = self.count_sides(group)
            print(f"{sides=}")
            som += area * sides

            if debug:
                self._print_group(group)

        return som

    def count_sides(self, group: set[tuple[int, int]]) -> int:
        sur = set()
        for r, c in group:
            sur |= {
                (rr, cc) for rr, cc in self.all_neigbors(r, c) if (rr, cc) not in group
            }
        print(f"{sorted(sur)=}")
        # count vertical
        prev_r = inf
        count_vertically = 0
        for r, c in sorted(sur):
            if r != prev_r:
                count_vertically += 1
            elif c - prev_c != 1:
                count_vertically += 1
            prev_r = r
            prev_c = c
        print(f"{count_vertically=}")
        # count horizontal
        print(f"{sorted(sur, key=lambda p: (p[1], p[0]))=}")
        prev_c = inf
        count_horizontally = 0
        for r, c in sorted(sur, key=lambda p: (p[1], p[0])):
            if c != prev_c:
                count_horizontally += 1
            elif r - prev_r != 1:
                count_horizontally += 1
            prev_r = r
            prev_c = c
        print(f"{count_horizontally=}")

        return count_horizontally + count_vertically

    def all_neigbors(self, row: int, col: int) -> Iterator[tuple[int, int]]:
        yield row - 1, col
        yield row, col + 1
        yield row + 1, col
        yield row, col - 1

    def _print_group(self, group: set[tuple[int, int]]) -> None:
        min_row, max_row = minmax(r for r, _ in group)
        min_col, max_col = minmax(c for _, c in group)
        print(sorted(group))
        print(f"{min_row, max_row=} and {min_col, max_col=}")
        print(
            "\n".join(
                "".join(
                    self.map[r][c] if (r, c) in group else "."
                    for c in range(min_col, max_col + 1)
                )
                for r in range(min_row, max_row + 1)
            )
        )
        print()


@test(tests=tests[:1])
def partII_test(s: str) -> int:
    g = GardenII(s)
    return g.price(True)


AAAA
BBCD
BBCC
EEEC
sorted(sur)=[(-1, 0), (-1, 1), (-1, 2), (-1, 3), (0, -1), (0, 4), (1, 0), (1, 1), (1, 2), (1, 3)]
count_vertically=4
sorted(sur, key=lambda p: (p[1], p[0]))=[(0, -1), (-1, 0), (1, 0), (-1, 1), (1, 1), (-1, 2), (1, 2), (-1, 3), (1, 3), (0, 4)]
count_horizontally=10
sides=14
[(0, 0), (0, 1), (0, 2), (0, 3)]
min_row, max_row=(0, 0) and min_col, max_col=(0, 3)
AAAA

sorted(sur)=[(0, 0), (0, 1), (1, -1), (1, 2), (2, -1), (2, 2), (3, 0), (3, 1)]
count_vertically=6
sorted(sur, key=lambda p: (p[1], p[0]))=[(1, -1), (2, -1), (0, 0), (3, 0), (0, 1), (3, 1), (1, 2), (2, 2)]
count_horizontally=6
sides=12
[(1, 0), (1, 1), (2, 0), (2, 1)]
min_row, max_row=(1, 2) and min_col, max_col=(0, 1)
BB
BB

sorted(sur)=[(0, 2), (1, 1), (1, 3), (2, 1), (2, 4), (3, 2), (3, 4), (4, 3)]
count_vertically=8
sorted(sur, key=lambda p: (p[1], p[0]))=[(1, 1), (2, 1), (0, 2), (3, 2), (1, 3), (4, 3), (2, 4), (3, 4)]
count_horizontally=6
sides=14
[(1, 2), (2, 2), (2, 3), (3, 3)]
min_row, max_row=(1, 3)

In [5]:
# print(f"Part II: {nr_stones_after_blining(puzzle, 75)}")
# GardenII(puzzle).price()

<link href="style.css" rel="stylesheet"></link>


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>259755538429618</code>.</p><p class="day-success">Both parts of this puzzle are complete! They provide two gold stars: **</p>

</main>
